In [ ]:
# Lab: Qiskit Runtime Primitives V2 - Hands-on Exploration
# Based on the Presentation: "Qiskit Runtime Primitives V2: Supercharging Your Quantum Programs"

# Welcome to the Qiskit Runtime Primitives V2 Lab!
# In this lab, you will get hands-on experience with Sampler V2 and Estimator V2,
# the core primitives of Qiskit Runtime. You will learn how to use them to execute
# quantum circuits, retrieve measurement data, and estimate expectation values efficiently.

# Let's get started!

# Setup - Import necessary Qiskit Runtime and standard Qiskit libraries
# Uncomment pip install if needed in your environment
#!pip install qiskit-ibm-runtime
import qiskit
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.quantum_info import SparsePauliOp

# Import Qiskit Runtime primitives and service
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, EstimatorV2, Session

print("Setup complete! Qiskit Runtime libraries imported.")

## Part 1: Introduction to Qiskit Runtime Primitives V2

In this lab, we'll be working with Qiskit Runtime Primitives V2.  As you learned in the presentation, these primitives offer an efficient way to execute quantum programs on IBM Quantum hardware.

We will explore two main primitives:

*   **Sampler V2:** For low-level execution and retrieving raw measurement outcomes (single-shot data).
*   **Estimator V2:** For high-level estimation of expectation values of observables.

Let's start with Sampler V2.

## Part 2: Sampler V2 - Exploring Measurement Outcomes

### Step 1: Load Runtime Service and Select Backend

First, we need to load your Qiskit Runtime Service account and select a backend to run our circuits on.

**Task 1:** Run the code cell below to initialize the Runtime Service and select a backend.

*Make sure you have configured your Qiskit Runtime Service account credentials. Replace `"YOUR_IBM_QUANTUM_SERVICE_INSTANCE"` and `"YOUR_BACKEND_NAME"` with your actual service instance and backend name if needed.*

In [ ]:
# Initialize Qiskit Runtime Service (replace with your instance and backend)
service = QiskitRuntimeService(instance="YOUR_IBM_QUANTUM_SERVICE_INSTANCE") # Replace with your instance
backend = service.backend("YOUR_BACKEND_NAME") # Replace with your backend name

print(f"Service initialized, using backend: {backend.name}")

### Step 2: Create and Transpile a Bell State Circuit

Let's create a simple Bell state circuit with measurement, similar to the example in the presentation. We'll then transpile it to be compatible with the backend we selected.

**Task 2:** Run the code cell below to create and transpile the Bell state measurement circuit.

In [ ]:
# Create a Bell circuit with measurement
bell_meas_circuit = QuantumCircuit(2, 2) # 2 qubits, 2 classical bits
bell_meas_circuit.h(0)
bell_meas_circuit.cx(0, 1)
bell_meas_circuit.measure([0, 1], [0, 1])

# Transpile the circuit for the selected backend
isa_bell_meas_circuit = transpile(bell_meas_circuit, backend)

print("Bell state measurement circuit created and transpiled.")
bell_meas_circuit.draw('mpl') # Draw the abstract circuit

### Step 3: Initialize Sampler V2 and Run the Circuit

Now we'll initialize the Sampler V2 primitive and run our transpiled Bell state circuit.

**Task 3:** Run the code cell below to initialize Sampler V2 and execute the Bell circuit.

*Observe the output.* We are running the circuit with `shots=10` and storing the `SamplerResult` in `sampler_result`.

In [ ]:
# Initialize Sampler V2
sampler = SamplerV2(backend=backend)

# Construct the pub (Primitive Unified Bloc) - for non-parametric circuit, it's just the circuit
pub_bell_meas = (isa_bell_meas_circuit,)

# Run the circuit using Sampler V2
sampler_job = sampler.run(pubs=[pub_bell_meas], shots=10)
sampler_result = sampler_job.result()

print("Sampler V2 job submitted and results retrieved.")

### Step 4: Explore Sampler V2 Results - `DataBin` and `BitArray`

Let's explore the results we got from Sampler V2. We'll access the `DataBin` and `BitArray` objects to see the measurement outcomes.

**Task 4:** Run the code cell below to explore the `SamplerResult`, `DataBin`, and `BitArray`.

*Examine the output.* The code will:

*   Access the `PubResult` object from `sampler_result`.
*   Get the `DataBin` object from `PubResult.data`.
*   Get the `BitArray` object from `DataBin.meas`.
*   Print various attributes of the `BitArray`: `shape`, `num_bits`, `num_shots`, and the raw `array` data.
*   Convert the `BitArray` to bitstrings and counts dictionary for easier interpretation.

In [ ]:
# Access PubResult and DataBin
pub_result_bell_meas = sampler_result[0] # Get the PubResult for the first (and only) pub
data_bin_bell_meas = pub_result_bell_meas.data
bit_array_bell_meas = data_bin_bell_meas.meas

print("BitArray Data Exploration:")
print(f"Shape: {bit_array_bell_meas.shape}")
print(f"Number of bits per shot: {bit_array_bell_meas.num_bits}")
print(f"Number of shots: {bit_array_bell_meas.num_shots}")
print(f"\nRaw array data (first few elements):\n {bit_array_bell_meas.array[:5]}") # Print first 5 elements

# Get bitstrings and counts for easier interpretation
bitstrings = bit_array_bell_meas.get_bitstrings()
counts = bit_array_bell_meas.get_counts()

print(f"\nBitstrings (first few):\n {bitstrings[:5]}")
print(f"\nCounts Dictionary:\n {counts}")

### Step 5: Run with Different Shots Values

Let's experiment with running the Sampler V2 with different numbers of shots. We can specify shots in the `sampler.run()` method.

**Task 5:** Run the code cell below to execute the Bell circuit with different shot values (100 and 1000 shots).

*Observe the output.* Notice how the `num_shots` attribute in the `BitArray` changes based on the `shots` value you provide in `sampler.run()`.

In [ ]:
# Run with shots=100
sampler_job_100 = sampler.run(pubs=[pub_bell_meas], shots=100)
result_100 = sampler_job_100.result()
bit_array_100 = result_100[0].data.meas
print(f"BitArray for shots=100 - Num Shots: {bit_array_100.num_shots}")

# Run with shots=1000
sampler_job_1000 = sampler.run(pubs=[pub_bell_meas], shots=1000)
result_1000 = sampler_job_1000.result()
bit_array_1000 = result_1000[0].data.meas
print(f"BitArray for shots=1000 - Num Shots: {bit_array_1000.num_shots}")

### Step 6: Explore Parametric Circuits with Sampler V2 (Optional)

**(Optional Task):** Let's briefly explore how Sampler V2 handles parametric circuits. We'll create a simple parametric circuit and run it with Sampler V2.

**Task 6 (Optional):** Run the code cell below to create a parametric circuit, define parameter values, and run it with Sampler V2.

*Observe the output and the shape of the `BitArray`.* Notice how the `BitArray` shape now reflects the shape of your `parameter_values` array.

In [ ]:
# Create a parametric circuit
param_circuit = QuantumCircuit(1, 1)
theta = Parameter('theta')
param_circuit.ry(theta, 0)
param_circuit.measure(0, 0)

# Transpile parametric circuit
isa_param_circuit = transpile(param_circuit, backend)

# Parameter values to evaluate
param_values = np.linspace(0, np.pi, 5) # 5 theta values from 0 to pi

# Construct pub with parametric circuit and parameter values
pub_param = (isa_param_circuit, param_values)

# Run with Sampler V2
sampler_job_param = sampler.run(pubs=[pub_param], shots=100)
sampler_result_param = sampler_job_param.result()
bit_array_param = sampler_result_param[0].data.meas

print("\nParametric Circuit BitArray Exploration:")
print(f"BitArray Shape for Parametric Circuit: {bit_array_param.shape}") # Shape should be (5,) - reflecting param_vals
print(f"Number of shots per parameter set: {bit_array_param.num_shots}")

## Part 3: Estimator V2 - Estimating Expectation Values

Now, let's move on to Estimator V2, which is designed for estimating expectation values.

### Step 7: Create a Bell State Circuit (No Measurement) and an Observable

For Estimator V2, our circuit should **not** include measurements. We will estimate the expectation value of an observable on the state prepared by this circuit. Let's use the Bell state circuit again, but without the measurement gates.  We'll also define a simple Pauli "ZZ" observable.

**Task 7:** Run the code cell below to create a Bell state circuit (no measurement) and the "ZZ" observable.

In [ ]:
# Create a Bell circuit WITHOUT measurement
bell_circuit_est = QuantumCircuit(2) # No classical bits needed
bell_circuit_est.h(0)
bell_circuit_est.cx(0, 1)

# Transpile the circuit for Estimator
isa_bell_est_circuit = transpile(bell_circuit_est, backend)

# Create a ZZ observable (SparsePauliOp)
zz_observable = SparsePauliOp(["ZZ"])

print("Bell state circuit (for Estimator) and ZZ observable created.")
bell_circuit_est.draw('mpl') # Draw the abstract circuit (no measurements)

### Step 8: Initialize Estimator V2 and Run for Expectation Value

Now we initialize the Estimator V2 primitive and run it with our Bell state circuit and "ZZ" observable.

**Task 8:** Run the code cell below to initialize Estimator V2 and estimate the expectation value of "ZZ" for the Bell state.

*Observe the output.*  The `EstimatorResult` will contain the estimated expectation value (`evs`) and its standard error (`stds`). For the Bell state Φ+ and the ZZ observable, the expectation value should be close to 1.

In [ ]:
# Initialize Estimator V2
estimator = EstimatorV2(backend=backend)

# Construct the pub for Estimator V2 - needs circuit and observables
pub_bell_est = (isa_bell_est_circuit, zz_observable)

# Run Estimator V2
estimator_job = estimator.run(pubs=[pub_bell_est])
estimator_result = estimator_job.result()

# Extract expectation value and standard error
expectation_value = estimator_result[0].data.evs
standard_error = estimator_result[0].data.stds

print("Estimator V2 job submitted and results retrieved.")
print(f"Estimated Expectation Value of <ZZ> for Bell State: {expectation_value:.4f} ± {standard_error:.4f}")

### Step 9: Explore Estimator V2 Results - `DataBin` (evs and stds)

Let's examine the `EstimatorResult` and `DataBin` from Estimator V2.

**Task 9:** Run the code cell below to explore the `EstimatorResult` and `DataBin`.

*Observe the output.* Notice how the `DataBin` for Estimator V2 contains `evs` and `stds` attributes, which are NumPy arrays representing the expectation values and standard errors.

In [ ]:
# Access PubResult and DataBin for Estimator
pub_result_est = estimator_result[0]
data_bin_est = pub_result_est.data

# Access evs and stds from DataBin
evs = data_bin_est.evs
stds = data_bin_est.stds

print("Estimator DataBin Exploration:")
print(f"Expectation Values (evs): {evs}")
print(f"Standard Errors (stds): {stds}")

### Step 10: Parametric Circuit & Multiple Observables with Estimator V2 (Optional)

**(Optional Task):** Let's try a more complex example with Estimator V2, using a parametric circuit and multiple observables (XX, YY, ZZ).

**Task 10 (Optional):** Run the code cell below to create a parametric circuit, define multiple observables, and run Estimator V2 to estimate their expectation values.

*Observe the output and the shape of `evs` and `stds`.* Notice how the shape now reflects both the parameter values (if you used them) and the number of observables.

In [ ]:
# Create a parametric Bell circuit (same as in Sampler example)
par_bell_circuit_est = QuantumCircuit(2)
theta_est = Parameter('theta')
par_bell_circuit_est.ry(theta_est, 0)
par_bell_circuit_est.cx(0, 1)
isa_par_bell_circuit_est = transpile(par_bell_circuit_est, backend)

# Parameter values for parametric circuit
param_vals_est = np.linspace(0, np.pi, 10) # 10 theta values

# Create multiple observables (XX, YY, ZZ) - as SparsePauliOps
observables_list = [SparsePauliOp(["XX"]), SparsePauliOp(["YY"]), SparsePauliOp(["ZZ"])]

# Construct pub with parametric circuit, observables, and parameter values
pub_param_est = (isa_par_bell_circuit_est, observables_list, param_vals_est)

# Run Estimator V2 with parametric circuit and multiple observables
estimator_job_param = estimator.run(pubs=[pub_param_est])
estimator_result_param = estimator_job_param.result()
data_bin_param_est = estimator_result_param[0].data

evs_param = data_bin_param_est.evs
stds_param = data_bin_param_est.stds

print("\nEstimator V2 Parametric Circuit & Multiple Observables Exploration:")
print(f"Expectation Values (evs) Shape: {evs_param.shape}") # Shape should be (3, 10) or (10, 3) due to broadcasting
print(f"Standard Errors (stds) Shape: {stds_param.shape}")

# You can further process and plot evs_param to visualize the expectation values for different observables and parameters

## Part 4: Explore Runtime Options (Challenges - Optional)

**(Optional Challenges):** For more advanced exploration, try experimenting with Runtime Options to control error mitigation and execution behavior.

### Challenge 1: Experiment with Twirling Options in Sampler V2

**Task (Challenge 1):** Modify the Sampler V2 initialization to enable Pauli Twirling and explore different twirling strategies.

*Refer to the presentation slides on "Twirling Options" and "Twirling Strategy" for available options.*

*Try different combinations of `enable_gates`, `enable_measure`, and `strategy` options in the `SamplerV2` constructor or by setting `sampler.options.twirling`.*

*Run the Bell state measurement circuit (or your parametric circuit) with twirling enabled and compare the results to the baseline without twirling.*

*Observe if twirling affects the measurement outcomes or reduces noise in your simulation results (you might need to simulate on a noisy simulator or real hardware to see a significant effect of error mitigation).*

In [ ]:
# Challenge 1 Code (Example - Experiment with Twirling Options in Sampler V2)

# Example 1: Enable gate twirling with 'active-circuit' strategy

twirling_sampler_1 = SamplerV2(backend=backend, options={"twirling": {"enable_gates": True, "strategy": "active-circuit"}})

# Run Bell circuit with twirling_sampler_1 and compare results to baseline sampler

# Example 2: Enable both gate and measure twirling

twirling_sampler_2 = SamplerV2(backend=backend, options={"twirling": {"enable_gates": True, "enable_measure": True}})

# Run Bell circuit with twirling_sampler_2 and compare results

# ... (Add your code to run simulations and compare results with different twirling options) ...

### Challenge 2: Experiment with Resilience Levels in Estimator V2

**Task (Challenge 2):** Modify the Estimator V2 initialization to explore different `resilience_level` options.

*Refer to the presentation slides on "Estimator Resilience Levels" and "Estimator Resilience Sub-Options" for available levels and customization options.*

*Try initializing `EstimatorV2` with different `resilience_level` values (0, 1, 2) and observe the impact on the estimated expectation values and standard errors.*

*For more advanced exploration, try customizing resilience options further using `estimator.options.resilience` and sub-options like `zne`, `pec`, etc.*

*Run the Bell state expectation value estimation (or your parametric circuit example) with different resilience levels and compare the results. You might need to run on a noisy simulator or real hardware to see the effects of error mitigation more clearly.*

In [ ]:
# Challenge 2 Code (Example - Experiment with Resilience Levels in Estimator V2)

# Example 1: Initialize Estimator with resilience_level=1

resilient_estimator_1 = EstimatorV2(backend=backend, options={"resilience_level": 1})

# Run Bell state expectation value estimation with resilient_estimator_1 and compare to baseline estimator

# Example 2: Initialize Estimator with resilience_level=2 and customize ZNE extrapolator

resilient_estimator_2 = EstimatorV2(backend=backend, options={"resilience_level": 2,
                                                               "resilience": {"zne": {"extrapolator": "linear"}}})


# Run Bell state expectation value estimation with resilient_estimator_2 and compare results

# ... (Add your code to run simulations and compare results with different resilience levels) ...

## Conclusion - Qiskit Runtime Primitives V2: Your Gateway to Efficient Quantum Computing

Congratulations on completing the Qiskit Runtime Primitives V2 Lab!  In this lab, you have:

*   Explored **Sampler V2** for low-level circuit execution and raw measurement data.
*   Explored **Estimator V2** for high-level expectation value estimation.
*   Learned how to use `Sampler.run()` and `Estimator.run()` with Primitive Unified Blocs (pubs).
*   Examined the structure of `SamplerResult` and `EstimatorResult` and how to access measurement data and expectation values.
*   (Optionally) Experimented with parametric circuits, multiple observables, and Runtime Options like Twirling and Resilience.

Qiskit Runtime Primitives V2 provide powerful tools for building and running efficient quantum programs on IBM Quantum hardware. By mastering these primitives and their options, you are well-equipped to tackle more complex quantum algorithms and applications!

**Further Resources:**

*   Qiskit Docs and Tutorials: [https://docs.quantum.ibm.com/](https://docs.quantum.ibm.com/)
*   Qiskit Primitives Docs: [https://docs.quantum.ibm.com/run/primitives](https://docs.quantum.ibm.com/run/primitives)
*   Qiskit Runtime API Docs: [https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/runtime_service](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/runtime_service)

**Thank you for participating in the lab! Keep building and exploring the quantum world with Qiskit Runtime!**